In [1]:
!pip install git+https://github.com/cayleypy/cayleypy

  Cloning https://github.com/cayleypy/cayleypy to /tmp/pip-req-build-g98ko1gd
  Running command git clone --filter=blob:none --quiet https://github.com/cayleypy/cayleypy /tmp/pip-req-build-g98ko1gd
  Resolved https://github.com/cayleypy/cayleypy to commit e005e674275114e49bf84c273d50623c4f2b0d28
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for cayleypy: filename=cayleypy-0.1.0-py3-none-any.whl size=121436 sha256=906dfa172bd6aa1ce4ccf5a68d515d9d03b232c5e2148fc94fe351ec4cc0bd38
  Stored in directory: /tmp/pip-ephem-wheel-cache-v406410q/wheels/5d/b5/53/86782f2010b218369465580e31a6c289b7f2a73390b39a23f3
Successfully built cayleypy


In [14]:
from cayleypy import PermutationGroups, CayleyGraph, Predictor, prepare_graph


graph32=CayleyGraph(PermutationGroups.lrx(32).with_central_state([0]*16+[1]*16))
X, y = graph32.random_walks(width=1000, length=250, mode="bfs")

In [15]:
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split


# Training parameters.
hidden_dims = [64]
learning_rate = 0.005


class Net(torch.nn.Module):
    def __init__(self, input_size, hidden_dims):
        super().__init__()

        layers = []
        for hidden_dim in hidden_dims:
            layers.append(torch.nn.Linear(input_size, hidden_dim))
            layers.append(torch.nn.GELU())
            input_size = hidden_dim
            
        layers.append(torch.nn.Linear(input_size, 1))
        self.layers = torch.nn.Sequential(*layers)

    def forward(self, x):
        #x = torch.nn.functional.one_hot(x.long(), num_classes=self.num_classes).float().flatten(start_dim=-2)
        return self.layers(x.float()).squeeze(-1)

input_size = graph32.definition.state_size
model = Net(input_size, hidden_dims).to(graph32.device)

# Prepare training and validation datasets.
val_ratio = 0.1
batch_size = 1024
dataset = TensorDataset(X, y.float())
val_size = int(len(dataset) * val_ratio)
train_size = len(dataset)-val_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_ds, batch_size=1024, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size)

loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epoch = [0]
def train_one_epoch():
    model.train()
    total_train_loss = 0
    for xb, yb in train_loader:
        pred = model(xb)
        loss = loss_fn(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item() * xb.size(0)

    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            pred = model(xb)
            loss = loss_fn(pred, yb)
            total_val_loss += loss.item() * xb.size(0)

    avg_train_loss = total_train_loss / train_size
    avg_val_loss = total_val_loss / val_size
    print(f"Epoch {epoch[0]} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")
    epoch[0] +=1

In [16]:
train_one_epoch()

Epoch 0 | Train Loss: 8576.5223 | Val Loss: 3943.3830


In [18]:
train_one_epoch()

Epoch 2 | Train Loss: 1452.5528 | Val Loss: 1142.8181


In [20]:
for _ in range(50):
    train_one_epoch()

Epoch 13 | Train Loss: 1056.6506 | Val Loss: 1044.3301
Epoch 14 | Train Loss: 1051.7265 | Val Loss: 1045.8320
Epoch 15 | Train Loss: 1047.7202 | Val Loss: 1034.7199
Epoch 16 | Train Loss: 1044.3524 | Val Loss: 1035.0812
Epoch 17 | Train Loss: 1041.3847 | Val Loss: 1026.8484
Epoch 18 | Train Loss: 1037.2906 | Val Loss: 1024.7876
Epoch 19 | Train Loss: 1034.1304 | Val Loss: 1022.9436
Epoch 20 | Train Loss: 1032.0311 | Val Loss: 1023.3606
Epoch 21 | Train Loss: 1030.7028 | Val Loss: 1023.8673
Epoch 22 | Train Loss: 1027.2247 | Val Loss: 1019.1542
Epoch 23 | Train Loss: 1024.3880 | Val Loss: 1012.9774
Epoch 24 | Train Loss: 1022.9767 | Val Loss: 1010.8688
Epoch 25 | Train Loss: 1020.4800 | Val Loss: 1010.0855
Epoch 26 | Train Loss: 1019.1123 | Val Loss: 1013.5585
Epoch 27 | Train Loss: 1017.4808 | Val Loss: 1006.4413
Epoch 28 | Train Loss: 1016.0414 | Val Loss: 1005.6515
Epoch 29 | Train Loss: 1014.4582 | Val Loss: 1004.9050
Epoch 30 | Train Loss: 1013.2301 | Val Loss: 1006.6377
Epoch 31 |

In [21]:
for _ in range(100):
    train_one_epoch()

Epoch 63 | Train Loss: 982.7765 | Val Loss: 982.7830
Epoch 64 | Train Loss: 984.4134 | Val Loss: 978.9164
Epoch 65 | Train Loss: 982.2243 | Val Loss: 983.1977
Epoch 66 | Train Loss: 981.2681 | Val Loss: 977.6310
Epoch 67 | Train Loss: 980.7106 | Val Loss: 976.4590
Epoch 68 | Train Loss: 980.0582 | Val Loss: 978.7453
Epoch 69 | Train Loss: 979.9697 | Val Loss: 980.9239
Epoch 70 | Train Loss: 980.0337 | Val Loss: 975.2041
Epoch 71 | Train Loss: 979.1939 | Val Loss: 980.2762
Epoch 72 | Train Loss: 977.8342 | Val Loss: 974.5480
Epoch 73 | Train Loss: 979.0592 | Val Loss: 981.5770
Epoch 74 | Train Loss: 977.7321 | Val Loss: 973.4996
Epoch 75 | Train Loss: 978.9430 | Val Loss: 977.5574
Epoch 76 | Train Loss: 977.7048 | Val Loss: 978.6529
Epoch 77 | Train Loss: 976.3333 | Val Loss: 972.6416
Epoch 78 | Train Loss: 977.3602 | Val Loss: 978.7624
Epoch 79 | Train Loss: 976.3525 | Val Loss: 972.6823
Epoch 80 | Train Loss: 975.8973 | Val Loss: 971.8137
Epoch 81 | Train Loss: 975.5642 | Val Loss: 97

In [29]:
import pandas as pd
from cayleypy import PermutationGroups, CayleyGraph, Predictor
import torch

baseline = pd.read_csv('/kaggle/working/ans0.csv')



GEN_MAP={"L":0, "R":1, "X":2}

def validate(graph, start_state, sol):
    result = graph.apply_path(start_state, sol)
    assert torch.equal(result[0], graph.central_state)
    #print("OK")

total_delta=0
total_score=0
ans=["permutation,solution"]
for i, row in baseline.iterrows():
    perm_str = row["permutation"]
    start_state = [int(x) for x in perm_str.split(",")]
    sol0 = [GEN_MAP[x] for x in row["solution"].split(".")]
    score0 = len(sol0)

    n = len(start_state)

    sol = sol0
    #print("n=", n)
    if n == 32:
        #graph = CayleyGraph(PermutationGroups.lrx(n).with_central_state([0]*(n//2) + [1]*(n//2)))
        bs_result=graph32.beam_search(start_state=start_state, predictor=Predictor(graph32, model),
                                      return_path=True, beam_width=200000, max_iterations=score0)
        assert bs_result.path_found
        sol = bs_result.path
        validate(graph32, start_state, sol) 
        delta = len(sol) - len(sol0)
        print(i, "n=", n, "score=", len(sol), "delta=", delta)
        total_delta+=delta
    else:
        pass

    total_score+=len(sol)
    sol_encoded = ".".join("LRX"[i] for i in sol)
    ans.append(f'"{perm_str}",{sol_encoded}')

print("total_delta=", total_delta)
print("total_score=", total_score)


100 n= 32 score= 258 delta= -103
101 n= 32 score= 219 delta= -78
102 n= 32 score= 207 delta= -87
103 n= 32 score= 275 delta= -114
104 n= 32 score= 197 delta= -92
105 n= 32 score= 287 delta= -140
106 n= 32 score= 256 delta= -111
107 n= 32 score= 244 delta= -85
108 n= 32 score= 303 delta= -132
109 n= 32 score= 278 delta= -131
110 n= 32 score= 209 delta= -96
111 n= 32 score= 333 delta= -165
112 n= 32 score= 268 delta= -140
113 n= 32 score= 289 delta= -129
114 n= 32 score= 237 delta= -118
115 n= 32 score= 233 delta= -72
116 n= 32 score= 293 delta= -123
117 n= 32 score= 266 delta= -120
118 n= 32 score= 223 delta= -76
119 n= 32 score= 280 delta= -133
120 n= 32 score= 273 delta= -140
121 n= 32 score= 239 delta= -83
122 n= 32 score= 284 delta= -116
123 n= 32 score= 236 delta= -108
124 n= 32 score= 220 delta= -72
125 n= 32 score= 230 delta= -64
126 n= 32 score= 253 delta= -91
127 n= 32 score= 219 delta= -56
128 n= 32 score= 298 delta= -131
129 n= 32 score= 221 delta= -87
130 n= 32 score= 185 de

In [30]:
with open("/kaggle/working/ans1.csv", "w") as f:
    f.write("\n".join(ans))